# KFold + Catboost Baseline + Advanced Feature Engineering
Накручиваем поверх Baseline продвинутое FE.

In [1]:
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

import joblib
import numpy as np
import matplotlib.pyplot as plt
import mlflow
import mlflow.sklearn
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import warnings

from catboost import CatBoostRegressor
from sklearn import set_config
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Вместо одного фиксированного разбиения на train/test используем стабильную стратегию кросс-валидации.
# Используем тут Cross-validation, потому что:
# 	•	нужно надёжно сравнить несколько разных моделей или гиперпараметров и понять, какая модель стабильнее и лучше в целом.
# 	•	хотим избежать случайных удач или провалов, связанных с конкретным разбиением на train/test.
# 	•	выбираем модель или гиперпараметры, которые потом будешь использовать для финального сабмишна на Kaggle.
# Делаем эту оценку, чтобы в дальнейших блокнотах-улучшениях сравнивать более корректно.
from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score, train_test_split

# Используем IterativeImputer:
# 	•	Он итеративно заполняет все пропуски сразу.
# 	•	Работает одновременно со всеми признаками, учитывая связи между ними.
# 	•	Не требует ручного управления порядком заполнения.
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from utils.data_manager import DataManager
from utils.model_manager import ModelManager
from utils.syth_generator_gaussian import CombinedSyntheticGenerator

In [2]:
# --- Глобально включаем вывод Pandas для всех трансформеров ---
# (Можно применять и к отдельным трансформерам/пайплайнам .set_output(transform="pandas"))
set_config(transform_output = "pandas")

In [3]:
dm = DataManager()
mm = ModelManager()

# Отключаем автологгирование, чтобы использовать ручное
mlflow.sklearn.autolog(disable=True)
warnings.filterwarnings("ignore", module="mlflow")  # Игнорируем предупреждения MLflow


In [4]:
RANDOM_STATE = 42
N_FOLDS = 5  # Например, 5 или 10

## 1. Загрузка данных

In [5]:
data_path = 'data/home-data-for-ml-course'
train_data = pd.read_csv(data_path + '/train.csv')
test_data = pd.read_csv(data_path + '/test.csv')
train_data.shape

(1460, 81)

## 2. Предобработка данных 

In [6]:
# Определение колонок для удаления
intuitively_bad_features = [
    'LotShape',  # Общая форма участка
    'LandContour',  # Рельеф участка
    'LotConfig',  # Конфигурация участка
    'LandSlope',  # Уклон участка
    'MiscFeature',
    'MiscVal',
]
bad_columns = dm.get_all_nan_cols(train_data)
bad_columns.append('Id')
bad_columns.extend(intuitively_bad_features)

In [7]:
# Разделение на X / y
X, y = dm.split_data_set_to_x_y(train_data, 'SalePrice')
print(X.shape, y.shape)
X_test = test_data.copy()
print(X_test.shape)

(1460, 80) (1460,)
(1459, 80)


In [8]:
X.drop(columns=bad_columns, inplace=True)
X_test.drop(columns=bad_columns, inplace=True)

In [9]:
def make_feature_eng_great_again(train_X_in, test_X_in):
    """Хелпер, который создает фичи, логарифмирует и выравнивает колонки."""
    # Работаем с копиями, чтобы не изменять оригинальные X, X_test вне функции
    train_X = train_X_in.copy()
    test_X = test_X_in.copy()

    # Словарь качественных признаков
    quality_dict = {'Ex': 5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, np.nan:0}

    def create_features(df):
        # Interactions (с проверкой на наличие колонок)
        if 'Neighborhood' in df.columns and 'MSZoning' in df.columns:
            df['Neighborhood_Zoning'] = df['Neighborhood'].astype(str) + '_' + df['MSZoning'].astype(str)
            # df.drop(columns=['Neighborhood', 'MSZoning'], inplace=True)  # mean CV RMSLE улучшился на 0.008, oof rmse улучшился на 0.0011; KAGGLE LB УПАЛ!!!
        if 'SaleType' in df.columns and 'SaleCondition' in df.columns:
            df['SaleType_Condition'] = df['SaleType'].astype(str) + '_' + df['SaleCondition'].astype(str)
            # df.drop(columns=['SaleType', 'SaleCondition'], inplace=True)  # ошибка увеличилась; KAGGLE LB УПАЛ!!!

        # Quality Score
        df['TotalQualScore'] = 0
        quality_cols = ['ExterQual', 'KitchenQual', 'BsmtQual', 'HeatingQC', 'GarageQual', 'FireplaceQu']
        for col in quality_cols:
            if col in df.columns:
                 df['TotalQualScore'] += df[col].map(quality_dict).fillna(0)
        # df.drop(columns=quality_cols, inplace=True)  # ошибка увеличилась; KAGGLE LB УПАЛ!!!

        # Porch/Deck Area and Flags
        df['PorchDeckArea'] = 0
        porch_cols = ['WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch']
        for col in porch_cols:
             if col in df.columns:
                df['PorchDeckArea'] += df[col].fillna(0)
        # df.drop(columns=porch_cols, inplace=True)  # Mean CV RMSE улучшился на 0.0008, oof rmse улучшился на 0.0006; KAGGLE LB УПАЛ!!!

        if 'Fireplaces' in df.columns:
            df['HasFireplace'] = (df['Fireplaces'] > 0).astype(int)
            # df.drop(columns=['Fireplaces'], inplace=True)  # !!! mean cv rmse улучшился на 0.0023, oof rmse улучшился на 0.0031. fold 3 проблемный очень хорошо улучшился без этой фичи; KAGGLE LB УПАЛ!!!
        if 'GarageType' in df.columns:
            df['HasGarage'] = (~df['GarageType'].isna()).astype(int)
            # df.drop(columns=['GarageType'], inplace=True)  # ошибка увеличилась; KAGGLE LB УПАЛ!!!
        if 'Fence' in df.columns:
            df['HasFence'] = (~df['Fence'].isna()).astype(int)
            # df.drop(columns=['Fence'], inplace=True)  # ошибка увеличилась; KAGGLE LB УПАЛ!!!
        df['HasPorchDeck'] = (df['PorchDeckArea'] > 0).astype(int)
        # df.drop(columns=['PorchDeckArea'], inplace=True)  # ошибка увеличилась; KAGGLE LB УПАЛ!!!

        return df

    def log_features(df, cols_to_log_list):  # Принимает СПИСОК колонок
        print(f"Applying log1p to: {cols_to_log_list}")
        for col_name in cols_to_log_list:
            if col_name in df.columns:
                # Добавим проверку на отрицательные значения перед логарифмированием
                if (df[col_name] < 0).any():
                     print(f"Warning: Column {col_name} contains negative values. Skipping log1p.")
                else:
                    df[col_name] = np.log1p(df[col_name])
            else:
                print(f"Warning: Column {col_name} not found in DF during log transform.")
        return df

    # 1. Создаем фичи
    train_X = create_features(train_X)
    test_X = create_features(test_X)
    print("Features created.")

    # 2. Определяем колонки для логарифмирования (ТОЛЬКО по трейну)
    numeric_cols = train_X.select_dtypes(include=np.number).columns
    skew_values = train_X[numeric_cols].skew()
    # Используем .index.tolist() чтобы получить список имен
    cols_to_log_list = skew_values[skew_values > 1].index.tolist()
    print(f"Columns identified for logging: {cols_to_log_list}")

    # # 3. Логарифмируем (используя ОДИН и тот же список)
    # train_X = log_features(train_X, cols_to_log_list)
    # test_X = log_features(test_X, cols_to_log_list)
    # print("Log transform applied.")

    # 4. Согласуем и сортируем колонки ПОСЛЕ всех манипуляций
    final_feature_cols = sorted(train_X.columns.tolist()) # Сортируем для стабильности
    train_X = train_X[final_feature_cols]
    test_X = test_X.reindex(columns=final_feature_cols, fill_value=0)
    print("Columns aligned and sorted.")

    return train_X, test_X

In [10]:
# Вызываем функцию с правильными данными (X, X_test)
X, X_test = make_feature_eng_great_again(X, X_test)

print("\nProcessing complete. Final shapes:")
print(f"X_processed: {X.shape}")
print(f"X_test_processed: {X_test.shape}")
print("\nExample processed X:")
print(X.head())

Features created.
Columns identified for logging: ['MSSubClass', 'LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'TotalBsmtSF', '1stFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtHalfBath', 'KitchenAbvGr', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PorchDeckArea', 'HasFence']
Columns aligned and sorted.

Processing complete. Final shapes:
X_processed: (1460, 81)
X_test_processed: (1459, 81)

Example processed X:
   1stFlrSF  2ndFlrSF  3SsnPorch Alley  BedroomAbvGr BldgType BsmtCond  \
0       856       854          0   NaN             3     1Fam       TA   
1      1262         0          0   NaN             3     1Fam       TA   
2       920       866          0   NaN             3     1Fam       TA   
3       961       756          0   NaN             3     1Fam       Gd   
4      1145      1053          0   NaN             4     1Fam       TA   

  BsmtExposure  BsmtFinSF1  BsmtFinSF2  ... ScreenPorch Street  TotRmsAbvGrd  \


In [11]:
# Получение числовых колонок
numeric_columns = X.select_dtypes(include=['float64', 'int64']).columns
# Получение нечисловых колонок (всех остальных)
non_numeric_columns = X.select_dtypes(exclude=['float64', 'int64']).columns

## 3. Обучаем модель с CV и корректируя данные

#### Нормализация данных через ColumnTransformer и Pipeline
В данном кейсе мы реализуем Заполнение числовых пропусков с помощью модели (Predictive imputation). Т.е. то, что пропущено в числовых признаках - будем заполнять не медианой или средним, а будем обучать модель, которая будет предсказывать пропуски (IterativeImputer + RandomForestRegressor)

In [12]:
# Создаем preprocessor с разными трансформерами для разных типов данных
# Числовые данные пропущенные предсказываем с помощью модели RandomForestRegressor

# Пайплайн для числовых признаков (итеративное заполнение)
numeric_transformer = Pipeline(steps=[
    ('imputer', IterativeImputer(
        estimator=RandomForestRegressor(n_estimators=50, random_state=RANDOM_STATE),
        max_iter=10,
        random_state=RANDOM_STATE
    )),  # Дает примерно +100 прирост качества vs mean/median
    # ('imputer', SimpleImputer(strategy='mean')),
    # ('scaler', StandardScaler())  # разницы не дает 
])

# Пайплайн для категориальных признаков (заполнение частым значением и кодирование)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# --- Объединяем препроцессоры ---
preprocessor = ColumnTransformer(
    transformers=[
        # Применяем к исходным числовым колонкам
        ('num', numeric_transformer, numeric_columns),
        # Применяем к исходным категориальным колонкам
        ('cat', categorical_transformer, non_numeric_columns)
    ],
    remainder='drop',   # 'passthrough' сохранит полиномиальные и другие колонки, которые не были ни числовыми, ни категориальными ИЗНАЧАЛЬНО
    verbose_feature_names_out=False  # Чтобы имена колонок не менялись на 'num__colname' и т.д.
)

default_params = {
    'iterations': 1000, 
    'learning_rate': 0.05, 
    'depth': 6, 
    'loss_function': 'RMSE', 
    'verbose': 0, 
    'random_seed': RANDOM_STATE,
    'early_stopping_rounds': 100  # ?
}
catboost_params = default_params


# Включить Early Stopping очень рекомендуется в параметрах. Это позволит модели на каждом фолде останавливаться тогда, когда метрика на валидационной части этого фолда (X_val, y_val_log) перестает улучшаться. Это самый надежный способ подобрать оптимальное число итераций для каждого фолда и избежать переобучения

# --- Финальный пайплайн ---
# Мы будем использовать preprocessor и модель отдельно в цикле CV
# для корректной работы early stopping с пайплайном sklearn. Поэтому нам финальный пайплайн - не нужен.
# final_pipeline = Pipeline([
#     ('preprocessing', preprocessor),
#     ('model', CatBoostRegressor(**catboost_params))
# ])

In [14]:
"""
Пояснения к тому, что тут происходит:

Тут мы ОЦЕНИВАЕМ и ТЕСТИРУЕМ подобранные фичи и гиперпараметры. В цикле бежим по N_FOLD-ам, бьем данные тренировочные в соотношении 80% на обучение, 20% на валидацию. Обучаем модель на трейне, валидируем на валидации. Смотрим на ошибки. Если по итогу нас все устраивает, мы дальше идем и обучаем модель с нуля на всех данных со всеми гиперпараметрами и фичами (да еще можно вычленить из цикла итерацию самую лучшую для нужного количества итераций обучения финальной модели),

oof_predictions — это массив, который в итоге будет содержать предсказания для каждого объекта из исходного тренировочного набора (X). Важно, что предсказание для конкретного объекта (например, дома №100) делается той моделью (из цикла CV), которая обучалась без этого объекта. Надежная оценка качества: OOF-предикты позволяют посчитать метрику качества (например, oof_rmse) на всем тренировочном наборе, при этом каждое предсказание было сделано "честно" (модель не видела этот объект при обучении). 
Эта оценка часто бывает более надежной, чем простое усреднение метрик по фолдам (mean_cv_rmse), так как она считается на полном наборе данных один раз. 
Можно сравнить oof_predictions с реальными значениями y_log (или y), чтобы понять, на каких объектах модель ошибается сильнее всего.
Стэкинг/Блендинг: OOF-предикты часто используются как новые признаки для обучения модели второго уровня (мета-модели) в ансамблях (стэкинг).
"""

# --- Кросс-Валидация ---
kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_STATE)

# # --- Кросс-Валидация с учетом временной структуры ---
# tscv = TimeSeriesSplit(n_splits=N_FOLDS, test_size=int(0.19*len(X)))

oof_predictions = np.zeros(X.shape[0])  # Для хранения out-of-fold предсказаний
fold_rmses = []
fold_best_iterations = []  # Будем сохранять лучшие итерации

mlflow.set_experiment("KFold Default With improved features")
with mlflow.start_run() as run:  # Сохраняем run для логирования артефактов
    for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)): 
        print(f"--- Fold {fold+1}/{N_FOLDS} ---")
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        # 1. Обучаем препроцессор ТОЛЬКО на трейне текущего фолда
        preprocessor.fit(X_train)

        # 2. Трансформируем трейн и валидацию
        X_train_prep = preprocessor.transform(X_train)
        X_val_prep = preprocessor.transform(X_val)

        # 3. Обучаем модель с early stopping
        model = CatBoostRegressor(**catboost_params)
        model.fit(X_train_prep, y_train,
                  eval_set=[(X_val_prep, y_val)],
                  verbose=0,  # Отключаем вывод, 100 - если хотим видеть обучение 
                 )

        # Сохраняем лучшую итерацию
        best_iter = model.get_best_iteration()
        fold_best_iterations.append(best_iter)
        print(f"Best iteration for fold {fold+1}: {best_iter}")

        # 4. Предсказания на валидации
        val_preds = model.predict(X_val_prep)
        oof_predictions[val_idx] = val_preds

        # 5. Оценка на фолде
        # Берем логарифмы, т.к. так оценивает Kaggle по условиям задачи
        fold_rmse = np.sqrt(mean_squared_error(np.log(y_val), np.log(val_preds)))
        print(f"Fold {fold+1} RMSE: {fold_rmse}")
        fold_rmses.append(fold_rmse)
        mlflow.log_metric(f"fold_{fold+1}_rmse", fold_rmse, step=fold+1)
        mlflow.log_metric(f"fold_{fold+1}_best_iter", best_iter, step=fold+1)
    
    # --- Итоговая оценка CV ---
    mean_cv_rmse = np.mean(fold_rmses)
    std_cv_rmse = np.std(fold_rmses)
    oof_rmse = np.sqrt(mean_squared_error(np.log(y), np.log(oof_predictions)))

    print(f"\nMean CV RMSE: {mean_cv_rmse:.4f} +/- {std_cv_rmse:.4f}")
    print(f"OOF RMSE: {oof_rmse:.4f}")
    print(f"Mean best iteration: {np.mean(fold_best_iterations):.0f}")

    # Логгирование итоговых метрик вручную
    mlflow.log_metric("mean_cv_rmse", mean_cv_rmse)
    mlflow.log_metric("std_cv_rmse", std_cv_rmse)
    mlflow.log_metric("oof_rmse", oof_rmse)
    mlflow.log_metric("mean_best_iteration", np.mean(fold_best_iterations))

--- Fold 1/5 ---
Best iteration for fold 1: 988
Fold 1 RMSE: 0.13268676307211907
--- Fold 2/5 ---
Best iteration for fold 2: 947
Fold 2 RMSE: 0.11501482412109708
--- Fold 3/5 ---
Best iteration for fold 3: 60
Fold 3 RMSE: 0.17362179092361715
--- Fold 4/5 ---
Best iteration for fold 4: 585
Fold 4 RMSE: 0.1257864222951497
--- Fold 5/5 ---
Best iteration for fold 5: 808
Fold 5 RMSE: 0.10139931418277556

Mean CV RMSE: 0.1297 +/- 0.0244
OOF RMSE: 0.1320
Mean best iteration: 678


In [18]:
# К чему стремимся
# --- Fold 1/5 ---
# Best iteration for fold 1: 988
# Fold 1 RMSE: 0.13268676307211907
# --- Fold 2/5 ---
# Best iteration for fold 2: 947
# Fold 2 RMSE: 0.11501482412109708
# --- Fold 3/5 ---
# Best iteration for fold 3: 60
# Fold 3 RMSE: 0.17362179092361715
# --- Fold 4/5 ---
# Best iteration for fold 4: 585
# Fold 4 RMSE: 0.1257864222951497
# --- Fold 5/5 ---
# Best iteration for fold 5: 808
# Fold 5 RMSE: 0.10139931418277556
# 
# Mean CV RMSE: 0.1297 +/- 0.0244
# OOF RMSE: 0.1320
# Mean best iteration: 678



In [15]:
# На прошлом шаге мы сделали KFOLD обучение, если довольны метриками, то получаем лучшую итерацию и обучаем финальную модель на ВСЕХ данных.

final_iterations = catboost_params['iterations']  # optuna нашла уже лучшие параметры
# Используем среднее реальных лучших итераций из CV
# final_iterations = int(np.mean(fold_best_iterations))
# Или можно попробовать медиану, она менее чувствительна к выбросам (как Fold 3)
# final_iterations = int(np.median(fold_best_iterations))
# final_iterations = best_final_iterations

print(f"\nTraining final model on all data with {final_iterations} iterations...")

final_catboost_params = catboost_params.copy()
final_catboost_params['iterations'] = final_iterations
final_catboost_params.pop('early_stopping_rounds', None)  # Убираем early stopping для финального обучения
print(f"Финальные параметры обучения модели: {final_catboost_params}")

final_model_pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('model', CatBoostRegressor(**final_catboost_params))
])

final_model_pipeline.fit(X, y)
print("Final model trained.")


Training final model on all data with 1000 iterations...
Финальные параметры обучения модели: {'iterations': 1000, 'learning_rate': 0.05, 'depth': 6, 'loss_function': 'RMSE', 'verbose': 0, 'random_seed': 42}
Final model trained.


In [16]:
# Предсказание на тест данных для сабмишна
final_test_pred_single_model = final_model_pipeline.predict(X_test)
submission_single = pd.DataFrame({'Id': test_data['Id'], 'SalePrice': final_test_pred_single_model})
submission_single.to_csv('final_KFold_Catboost_Baseline.csv', index=False)
submission_single.head()

,Id,SalePrice
0,1461,121982.723142
1,1462,157490.604484
2,1463,189784.296524
3,1464,192438.774384
4,1465,176565.159645


In [17]:
# Kaggle BASELINE

# 12879.10346